# **Finding Lane Lines on the Road** 



### Pipeline 
The pipeline implemented here consists of 11 steps, described below. 

#### 1. Alter image saturation and contrast
As in the 'challenge' video, yellow lane lines can be difficult to identify in bright sunlight, and on irregular road surfaces which are not a convenient shade of black. Increasing the saturation of the image enables easier detection of yellow lane lines. Coupled with a contrast adjustment, it becomes possible to recognize the yellow lane lines in the challenge video during the two-second interval of bright sunlight, $~0:04$ to $~0:06$. 

#### 2. Convert to grayscale 
To enable the detection of edges, the image must be converted to grayscale, as described in the accompanying lessons. 

#### 3. Apply a Gaussian filter  
A gaussian blur filter with a kernel of size 3 is applied as a means of noise reduction, and to ensure that the most prominent lines in the image are more easily detected in the following steps. 

#### 4. Canny edge detection 
The Canny transform is applied to the image, indicating the pixels where the image gradient is above a certain threshold. 

#### 5. Define region of interest (ROI) 
The region of interest is statically defined, as a trapezoid whose two longest edges are approximately parallel to the left, and right lane lines respectively. 

#### 6. Detection of line segments via the hough transform 
Applying the hough transform, line segments are detected. Line segments are then seperated by the lane to which they appear to belong, based on their slope, $m$. Recalling that the origin is in the upper left corner of the frame, we define lines with $m > 0$ to be part of the right lane, and lines with $m \leq 0$ to be part of the left lane. **Each step below marked with an (L/R) is performed separately for each lane line.**

#### 7. Line segments with outlier slopes are excluded. (L/R)
Prior to the definition of a continuous lane line, segments whose slopes are deemed to be outliers are removed. First, a soft maximum and minimum threshold are defined, call these $m_{max}$ and $m_{min}$. Second, compute the interquartile range (IQR) of all line segments with slopes, $m$ such that $m_{min} < m < m_{max}$. 

Let $Q1$ be the lower quartile of this restricted distribution of slopes in the allowed rasnge, and let $Q3$ be its upper quartile. 

Finally, any segment whose slope falls outside the range $[Q1 - 1.5 \times IQR, Q3 + 1.5 \times IQR]$, is defined to be an outlier, and is excluded from further processing. Note that there is no explicit exclusion of slopes greater than $m_{max}$ or less than $m_{min}$. Rather we model outliers based on a restricted range of slopes, and *only* remove segments based on the prooperties of this distribution. This effectively turns $m_{max}$ and $m_{min}$ into soft thresholds, enabling more organic outlier removal. 

#### 8. Linear modelling of lane lines (L/R)
Using the collection of endpoints of all segments deemed *inliners* in step **7**, a linear model is fit for each lane via simple linear regression. 

#### 9. Exponential smoothing of lane parameters (L/R)
To reduce jitteriness, take into account past knowledge of the location of lane lines,  and render the lane line model more robust, exponential smoothing is applied to the parameters of the linear fit described in **8**. Let $m_i, b_i$ be the slope and y-intercept of the linear model of a lane line computed in **8**, for frame $i$ of a video. We define $m_{i,smooth}, b_{i,smooth}$ as follows: 

$m_{i,smooth} = \gamma m_i + (1 - \gamma) m_{i-1, smooth}$

$b_{i,smooth} = \gamma b_i + (1 - \gamma) b_{i-1, smooth}$

where $\gamma$ is a smoothing coefficient. Best results were achieved here for $\gamma = 0.1$. This makes the lane modelling procedure heavily reliant on past results, and extremely conservative. While this is advantageous in cases where difficult conditions may momentarily impede lane-line identification, and works quite well for the videos provided here, realistically it would make adapting to rapid changes difficult.

**Note** The smoothing procedure is only valid for video. When the pipeline is applied to single images, no smoothing takes place. 

#### 10. Drawing of lane lines. (L/R)
The maximum and minimum vertical coordinates, $y_{min}, y_{max}$, of any line segment from step **6** are computed. 

Let $m, b$ be the (smoothed, if applicable) slope and (smoothed, if applicable) y-intercept of a linear model defined in **8,9**. A straight line is drawn from $(x_0, y_{min})$ to $(x_1, y_{max})$ for each lane, where $y_{min}= mx_0 + b$, and $y_{max} = mx_1 + b$.

#### 11. Overlay the lane lines on the original image. 
The lane lines are drawn on the grayscale working image which has been heavily processed as part of the edge detection procedure. This working copy is combined with the original input image, enabling the input image to be reproduced with the additional lane lines drawn over it. 

## Shortcomings and Possible Improvements 

### Runtime 
One must imagine that a procedure for detecting lane lines will run in concert with many other simultaneous tasks on an autonomous vehicle. At present the pipeline is implemented with a single thread of execution. An updated pipeline, with more support for concurrent execution, perhas GPU support, particularly where operations on images are per-pixel, would be more efficient. 

### Obstacles, weather conditions, road conditions 
An approach to lane line detection based solely on vision and following the methodology  implemented here, is severly limited in all but near-ideal conditions. 

First, on a wet or icy road, where lane lines may be visually broken and the road very reflective, edge detection will undoubtedly be far more difficult. 

Obstacles on the road, proximal vehicles, and irregularl lane lines would all cause the proposed pipeline to faulter. 

Where road work is being performed, it is common for lane lines to be painted over in black, and new lanes defined alongside. This process creates spurious edges, which are difficult to distinguish from genuine lane lines. The use of different qualities of paint will present further challenges.

On many roads, lane lines are not omnipresent. Indeed, one must stress that the videos provided, which depict *a nice day in California on a highway with almost no traffic* are a wildly simplified case, and totally unrepresentative of the real challenge of identifying lane lines, or developing lane-keeping methods for use "in the wild." 

To make the pipline more robust in the face of the aforedescribed averse conditions, a first step would be more extensive image pre-processing, for the removal of glare and reflection. A hardware solution may also be necessary: a front facing camera is the equivalent of putting blinders on a horse, the detection procedure could be augmented by data from cameras positioned in the wheelarches of the vehicle. Specialized hardware filters (e.g. a *polarizing filter* fitted to the camera 

A human driver has a much richer notion of what a lane line represents, which enables safe-driving in averse conditions, and when there are no lane lines at all. Indeed, roads without lane lines should become more common, as it's been noted that a reduction in signage and  improves both the flow of traffic, and leads to fewer accidents [See here](https://www.theguardian.com/commentisfree/2016/feb/04/removal-road-markings-safer-fewer-accidents-drivers). 

### Curved lane lines 
A very obvious limitation of this simplistic approach is an innability to model curved lane lines in, for example, tight bends. Assuming the same ideal conditions in the provided videos, however, this can be very easily dealt with. 

First, a simple alternative would be fitting a bezier curve to each lane, rather than a linear model, which is a self-evident oversimplification. While a linear model was sufficient for the sake of completing the project in limited time, more suitable modelling approaches are available. 

Alternatively, the vertical position of line segments could be binned, and a linear model fit within each bin. Approaches abound, and I look forward to playing with this when more spare time sneaks up on me, or later in the course, as I assume the topic will present itself. 

### Inability to automatically adapt to unexpected conditions 
Lane line detection as implemented here cannot adapt to unexpected conditions, and the approach lacks a dynamic quality. Aside from what has been described above, the vehicle itself may be moved out of a central position. 

In all examples, the car is driving steadily, at a constant speed, in a single lane. No maneuvres are performed. A much more difficult task, is the detection of lane lines regardless of the path taken by the vehicle. 

In such cases it may, for instance, be more appropriate to detect all lanes on the road, rather than restricting attention to the markings immediately in front of the vehicle. The approch applied here and in the lessons is akin to putting blinders on the vehicle. More appropriate decisions can then be made by the models responsible for vehicle function and movement, in difficult situations. 

### Threshold based-approaches
Finally, nearly all the algorithms applied in the pipeline as implemented here, and in the lessons, are based on thresholds. This *line in the sand* appraoch, is generally best replaced with a probabilistic method, where decisions can be made based on a computed degree of confidence in an estimate, rather than an arbitrary threshold. 

### Learning for adaptation 
Rather than vision alone, a hybrid solution relying on an underlying machine learning infrastcuture would, in many cases, render the detection pipeline more robust, and more adaptable to averse conditions. This would lead to a more adaptable model, and allow valuable measures of statistical confidence. 

However, a word of caution: it would be wildly irresponsible to rely solely on a machine learning solution in a mission critical, and life critical system. A hybrid approach is key, as any statistical model is, realisticaly, imperfect. One can imagine an emergency system which will take over and guide the vehicle safely to a stop, or safely enable the intervention of a human driver, in cases where the decisions being made by the learned model do not meet certain confidence requirements in exceptional sitautions. 


## A final note on implementation choices 
An effort has been made to adhere to the schema suggested by the lessons, and by the sample code and helper methods provided in this notebook. 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Additional imports 
from collections import deque 
import math 
import os

from moviepy.editor import VideoFileClip
from IPython.display import HTML


## Helper methods (original/modified)

The methods below were provided with the project outline. First, `hough_lines`, `draw_lines`, have been extended to support an additional paramter `smoother`. This parameter is to be an instance of `LaneSmoother` (see above).  

Next, `draw_lines` has been extended to include 

1. Drawing of consensus, smoother lane markers, rather than individual line segments 

2. Removal of segments whose slopes are outliers or whose slopes do not fall within the range expected for valid lane lines.  

3. Linear modelling of lane lines based on the extremeties of the segments computed in `hough_lines` 

4. Exponential smoothing of the parameters of linear lane-line models. 

A more in-depth discussion follows in the reflection. 

In [ ]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_innterest(img, vertices):
    """
    Applies an image mask.

    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)

    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    #filling pixels inside the polygon defined by "vertices" with the fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=10, smoother=None):
    """ Draw lane lines, with removal of outlier slopes, and exponential smoothing.

    img (Image)   : Current working copy of the frame/image (see `hough_lines`).

    lines (array) : Extremeties of line segments computed in `hough_lines`.

    color         : List of length 3, each entry must be in [0,255]. Specifies
    the RGB color for drawing lane lines.

    thickness     : Desired line thickness.

    smoother      : An instance of LaneSmoother (see LaneSmoother). This object
    holds and updates the parameters of the linear models of lane lines, applying
    exponential smoothing.


    Lane line segments are processed as follows:

    (1) The are separated into right lanes (RL), and left lanes (LL) based
    on the slope of the segments.

    (2) Lane segments with outlier slopes are detected and excluded.

    (3) Using the extremities of each lane segment, a linear model is fir
    to the lane.

    (4) If the application is video, and if an instance of LaneSmoother is provided,
    exponential smoothing based on lanes detected in previous frames
    is applied to the parameters of the model.

    Returns:
      None. Draws directly to the input image (`img`).
    """

    ## For readability, define a few useful abstractions
    f_linear_solve = lambda y, fit: math.floor((y-fit[1])/fit[0])
    linear_model   = lambda x, y: np.polyfit(x=x, y=y, deg=1)

    lines_flat = flatten(lines.tolist())
    lane_params = list(map(line_params, lines_flat))

    ## General constants
    ymax = img.shape[0]
    ymin  = min([min(y1, y2) for (x1,y1,x2,y2) in lines_flat])

    ## The line segments corresponding to each lane are processed seperately. The lane
    ## to which a segment belongs is determined by its slope. **Recalling that the origin
    ## is in the upper left corner of the frame**, one may adopt the convention that
    ## segments of the right lane have positive slope, while segments of the left
    ## lane have negative slope.
    ##
    ## The right lane (positive slope) is processed first.

    right        = [m > 0 for m,b in lane_params]
    right_params = [(m,b) for i,(m,b) in enumerate(lane_params, 0) if right[i]]
    right_out    = moutliers([m for m,b in right_params], 0.3, 1)

    right_lines  = [line for i,line in enumerate(lines_flat, 0) if right[i]]
    right_lines  = [line for i,line in enumerate(right_lines, 0) if not right_out[i]]

    right_x = flatten([[x1,x2] for (x1,y1,x2,y2) in right_lines])
    right_y = flatten([[y1,y2] for (x1,y1,x2,y2) in right_lines])

    right_lane = linear_model(right_x, right_y)

    if smoother:
        right_lane = smoother.smooth(right_lane[0], right_lane[1], True)

    cv2.line(img,
             (f_linear_solve(ymax, right_lane), ymax),
             (f_linear_solve(ymin, right_lane), ymin),
             color, thickness)


    left        = [m <= 0 for m,b in lane_params]
    left_params = [(m,b) for i,(m,b) in enumerate(lane_params, 0) if left[i]]
    left_out    = moutliers([m for m,b in left_params], -1, -0.3)

    left_lines  = [line for i,line in enumerate(lines_flat, 0) if left[i]]
    left_lines  = [line for i,line in enumerate(left_lines, 0) if not left_out[i]]

    left_x = flatten([[x1,x2] for (x1,y1,x2,y2) in left_lines])
    left_y = flatten([[y1,y2] for (x1,y1,x2,y2) in left_lines])

    left_lane = linear_model(left_x, left_y)

    if smoother:
        left_lane = smoother.smooth(left_lane[0], left_lane[1], False)

    cv2.line(img,
             (f_linear_solve(ymin, left_lane), ymin),
             (f_linear_solve(ymax, left_lane), ymax),
             color, thickness)


def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, smoother=None):
    """ Extract and draw hough lines.

    Args:
    img (Image): The output of a Canny transform.

    smoother (LaneSmoother): Insance of LaneSmoother, or None. If None, we do not perform
    any smoothing. Smoothing is only intended for video, where the linear models for the lane lines
    visible in a given frame are updated based on the lanes detected in the previous frames.
    For applications to single images, the value of `smoother` should be None, as smoothing
    cannot be applied.

    Returns:
    An image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)

    draw_lines(line_img, lines, smoother=smoother)
    return line_img


def weighted_img(img, initial_img, α=0.8,  β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.

    `initial_img` should be the image before any processing.

    The result image is computed as follows:

    initial_img * α + img * β + λ
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)



## Helper Sources (additional)
The following methods (and one class definition) have been added, and are not present in the original project outline. They are presented separately here for clarity. Below, you will find the following: 

1. `moutliers` A helper method for flagging outliers in a set of slopes based on the interquartile range of the distribution of slopes which fall between pecified minimum and maximum allowed values. 

2. `line_params` A helper method for computing the parameters of a line segment, given its endpoints. 

3. `flatten` A trivial helper method, for 'flattening' a list, one level deep (i.e. `[[1,2]]` becomes `[1,2]`). 

4. `LaneSmoother` Class whose responsibility is the application of exponential smoothing to the parameters of the linear lane-line model. See the class documentation for more information. 

In [ ]:
def moutliers(values, min_allowed, max_allowed):
    """ Determine the outliers of a set of slopes.

    Consider the distribution of slopes which fall
    in the range [min_allowed, max_allowed].

    Compute the interquartile range (IQR) of this
    **restricted distribution** of slopes, and
    define any slope that falls outside the range

    [Q1 - 1.5(IQR) , Q3 + 1.5(IQR)]

    as an outlier, where Q1 is the lower quartile, and Q3
    is the upper quartile.

    This method effectively turns min_allowed, max_allowed
    into 'soft' limits, which is more natural given the
    application.

    Args:
    values (List) : List of slope values to test.
    min_allowed (Number) : The minimum allowed slope for the provided collection.
    max_allowed (Number) : The maximum allowed slope for the provided collection.

    Returns:
    A list of booleans, indicating whether or not the
    corresponding position in the input is an outlier. True indicates
    an outliers, False indicates an inliner.
    """
    quartiles = lambda x: np.percentile(x, np.arange(0, 100, 25))
    iqr = lambda x: abs(x[2] - x[1])
    in_range = lambda x: x > min_allowed and x < max_allowed

    ## Compute quartiles based ONLY on the values within the allowed range.
    values_q = quartiles([v for i,v in enumerate(values, 0) if in_range(values[i])])
    values_q_lower = values_q[1] - 1.5*iqr(values_q)
    values_q_upper = values_q[2] + 1.5*iqr(values_q)

    return [(v < values_q_lower or v > values_q_upper) for v in values]



def line_params(l):
    """ Compute the parameters of the line l.

    Args:
    l (List): The elements, in order, are x1,y1,x2,y2

    Returns:
    Tuple, (m, b) the first element is ((y2-y1)/(x2-x1)),
    the second is the value of the y-intercept.
    """

    x1,y1,x2,y2 = l
    m = 1e-5 if np.isclose((y2-y1),0.) or np.isclose((x2-x1), 0.) else ((y2-y1)/(x2-x1))
    b = y1 - m*x1
    return (m,b)

def flatten(L):
    """
    Flatten the list L (one level).
    """
    return [elem for l in L for elem in l]

class LaneSmoother:
    """ Exponential smoothing of the parameters of the lane line.

    Lane lines are modelled as a linear function of x-coordinate
    in the image. This class applies simple exponential smoothing to
    the parameters. Parameters are smoothed as:

    m1 = gamma*m1 + m0*(1 - gamma)
    b1 = gamma*b1 + b0*(1 - gamma)


    Attributes:
    left_prev (Dict): A dictionary {m: ...,b: ...} of the previous smoothed parameters. m is the
    slope of the lane line, and b is the y-intercept.

    right_prev (Dict): As above, but for the right lane.
    """
    def __init__(self, gamma):
        self.gamma = gamma

        self.left_prev  = {'m': None, 'b': None}
        self.right_prev = {'m': None, 'b': None}

    def smooth(self, new_m, new_b, left):
        """ Compute smoothed parameter estimates and update previously stored estimate.

        The main working method of the LaneSmoother, class, `smooth` takes
        a 'raw' slope and a 'raw' y-intercept, and applies exponential smoothing
        with smoothing factor gamma. Note that the smoothing factor is supplied
        at instantiation of the LaneSmoother class, and is constant throughout the
        life of the object. New parameter estimates are computed as:

        x1_smooth = (gamma * x1_orig) + (1- gamma)*x0

        where x1_smoother is the updated smoothed parameter, x1_orig is the
        parameter before smoothing, and x0 is the previous (smoothed) value
        of the parameter.

        Args:
        new_m (Number)  : Slope to be smoothed.
        new_b (Number)  : Y-intercept to be smoothed.
        left  (Boolean) : Right or left lane?

        Returns:
        Smoohted parameter estimates for the slope, m,  and y-intercept, b, of the model,
        as a tuple (m,b).
        """

        this_prev = self.left_prev if left else self.right_prev

        if this_prev['m'] and  this_prev['b']:
            this_prev['m'] = new_m * self.gamma + (1 - self.gamma)*this_prev['m']
            this_prev['b'] = new_b * self.gamma + (1 - self.gamma)*this_prev['b']
        else:
            this_prev['m'] = new_m
            this_prev['b'] = new_b

        return (this_prev['m'], this_prev['b'])



def image_saturation(image, saturation_factor):
    """ Change the saturation of an RGB image.

    Args:
    image (Image) : The image whose saturation we wish to change. Must be
    cv2.RGB

    saturation_factor (Number) : Factor by which we wish to change saturation.

    Note that this method works by converting the image from RGB to HSV space
    using opencv's cvtColor method.

    Returns:
        Image with modified saturation.
    """
    image_working = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    (h, s, v) = cv2.split(image_working)
    s = s*saturation_factor
    s = np.clip(s,0,255)
    image_working = cv2.merge([h,s,v])
    image_working = cv2.cvtColor(image_working, cv2.COLOR_HSV2BGR)
    return(image_working)



# Pipeline
More details are provided in the reflection. See the commentary below for an outline. 

In [ ]:
def process_image(image, smoother):
    """ Detect and mark lane lines in an image.

    Args:
    image (Image) : An single image to process.

    smoother (LaneSmoother) : Either an instance of LaneSmoother or None. If this
    processing pipeline is being applied to single images, smoothing is not supported.


    The pipeline is as follows:

    (1) Increase image saturation by a factor of 2.
    (2) Increase image contrast (alpha 1.0, beta -100)
    (3) Convert to grayscale
    (4) Guassian blur
    (5) Canny edge detection
    (6) Define region of interest
    (7) Detect line segment via the Hough transform.
       (7.1) Detect and exclude line segments deemed to be outliers for each lane
       (7.2) Apply a linear model to each lane
       (7.3) If applicable, smooth the parameters of the model describing each lane based
             on previous iterations.
    (8) Combine the overlay, where lane lines have been drawn, and the original image.

    Returns:
        A numpy.ndarray representing an image with marked lane lines.
        Dimensions are identical to input image, color is preserved.
    """

    image_working = image

    xsize = image_working.shape[1]
    ysize = image_working.shape[0]

    ## Up saturation
    image_working = image_saturation(image_working, 2)


    ## Modify the contrast of the image. With the saturation
    ## increase above ^ this makes yellow lane lines more
    ## easily spotted in bright sunlight (easily observed
    ## in the challenge video).
    beta  = np.array([-100.0])
    cv2.add(image_working, beta, image_working)


    image_working = grayscale(image_working)

    image_working = gaussian_blur(image_working, kernel_size=3)

    image_working = canny(image_working, low_threshold=50, high_threshold=150)

    ## The image here is considered as a 32 x 32 grid, to make it simpler to
    ## reason about the space being defined as the ROI.
    roi_vertices  = np.array([[(xsize*(3/32), ysize),
                               (xsize*(15/32), ysize*(19/32)),
                               (xsize*(17/32), ysize*(19/32)),
                               (xsize, ysize)]], dtype='int32')

    image_working = region_of_interest(image_working, vertices=roi_vertices)

    image_working = hough_lines(image_working, rho=1, theta=1*(math.pi/180),
                                threshold=15, min_line_len=30, max_line_gap=30,
                                smoother=smoother)

    result = weighted_img(image_working, image)

    return result



## Tests on Images
Images are saved in the `test_images` directory, please view them there. 


In [ ]:
test_images = ['solidWhiteRight.jpg', 'solidYellowCurve.jpg',
               'solidWhiteCurve.jpg', 'solidYellowLeft.jpg',
               'solidYellowCurve2.jpg', 'whiteCarLaneSwitch.jpg']

for image_path in test_images:
    print("Processing " + image_path + "\n")
    image = mpimg.imread('test_images/' + image_path)
    res = process_image(image, smoother=None)
    mpimg.imsave('test_images/test_' + image_path, res)



## Test on Videos


In [ ]:
white_output = 'white.mp4'
smoother = LaneSmoother(0.1)
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(lambda frame: process_image(frame, smoother=smoother))

%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [ ]:
yellow_output = 'yellow.mp4'
smoother = LaneSmoother(0.1)
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(lambda frame: process_image(frame, smoother=smoother))

%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Optional Challenge

The pipeline is applied to all three vidoes here, however, the utility of the removal of outlier slopes is most evident in the processing of this challenge video, where line segments detected on the hood of the vehicle are excluded. See this reflection above. 

Increasing the saturation and contrast of each image to compensate for the effect of bright sunlight and an irregular road surface from 0:04 - 0:06 assisted in the detection of the yellow lane line for these portions of the image. 

Finally, applying exponential smoothing to the parameter of the linear model used to describe lane lines eliminated a large part of the jitteriness that would otherwise be observed in all provided videos. 

In [ ]:
challenge_output = 'extra.mp4'
smoother = LaneSmoother(0.1)
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(lambda frame: process_image(frame, smoother=smoother))

%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))